In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader,random_split

import torchvision.models as models
from torch import nn
from torch.nn import functional as F
from torch import optim
import os
from torchvision import transforms,datasets

In [2]:
tf = transforms.Compose([
    transforms.Resize([112,112]),            #改变形状
    transforms.ToTensor(),            # 将numpy数组或PIL.Image读的图片转换成(C,H, W)的Tensor格式且/255归一化到[222,1.222]之间
])

In [3]:
dataset = torchvision.datasets.LFWPeople(root="LFWPeople",transform=tf,download=True)

Files already downloaded and verified


In [4]:
print(len(dataset))

13233


In [5]:
train_size=int(len(dataset)*0.8)
test_size=len(dataset)-train_size

In [6]:
train_dataset,test_dataset=random_split(dataset,[train_size,test_size])

In [7]:
train_loader=DataLoader(train_dataset,batch_size=32,num_workers=0,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=32,num_workers=0,shuffle=False)

In [8]:
print(len(train_loader))
print(len(test_loader))

331
83


In [9]:
class Arcsoftmax(nn.Module):
    def __init__(self, feature_num, cls_num):
        super().__init__()
        self.w = nn.Parameter(torch.randn((feature_num, cls_num)),requires_grad=True)   #nn.Parameter将一个不可训练的类型Tensor转换成可以训练的类型parameter并将这个parameter绑定到这个module里面
        self.func = nn.Softmax()                                                        #二分类
 
    def forward(self, x, s=64, m=0.5):                                                  #s=64为超参数m为弧度
        x_norm = F.normalize(x, dim=1)
        w_norm = F.normalize(self.w, dim=0)                                             #传入的参数nn.Parameter在0维上进行标准化
 
        cosa = torch.matmul(x_norm, w_norm) / s                                         #torch.matmul二维的点成，高维的矩阵乘法
        a = torch.acos(cosa)
 
        arcsoftmax = torch.exp(
            s * torch.cos(a + m)) / (torch.sum(torch.exp(s * cosa), dim=1, keepdim=True) - torch.exp(
            s * cosa) + torch.exp(s * torch.cos(a + m)))                                #代码实现公式
 
        return arcsoftmax

In [10]:
class FaceNet(nn.Module):
    def __init__(self):
        super(FaceNet, self).__init__()
        self.sub_net = nn.Sequential(
            models.mobilenet_v2(pretrained=True)     #pretrained=True为加载预训练模型     #导入mobilenet_v2
            #models.mobilenet_v2(pretrained=True)
        )
        self.feature_net = nn.Sequential(
            nn.BatchNorm1d(1000),
            nn.LeakyReLU(0.1),                                      #0.1指的是leakRelu负半轴的倾斜角
            nn.Linear(1000, 512, bias=False),
        )
        self.arc_softmax = Arcsoftmax(512, 10574)                     #112和最终的分类的数量有关，有多少个人就分多少个类
 
    def forward(self, x):
        y = self.sub_net(x)                                         #y是原本的mobilenet_v2()的输出值
        feature = self.feature_net(y)                               #self.feature_net网络导数第二层
        return feature, self.arc_softmax(feature)                   #前向推理返回的是特征和arc_softmax分类
 
    def encode(self, x):
        return self.feature_net(self.sub_net(x))                    #返回的是倒数第二层的值

In [11]:
def compare(face1, face2):
    face1_norm = F.normalize(face1)                                 #对传入的人脸进行标准化
    face2_norm = F.normalize(face2)
    cosa = torch.matmul(face1_norm, face2_norm.T)                   #矩阵乘法
    # cosb = torch.dot(face1_norm.reshape(-1), face2_norm.reshape(-1))
    return cosa

In [12]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
if __name__ == '__main__':
    # 训练过程
    
    net = FaceNet().to(device) #网络实例化并传入cuda
    
    # save_path=r'params/face_discern.pt'
    # if os.path.exists(save_path):
    #     net.load_state_dict(torch.load(save_path))
    # else:
    #     print("No Param")
 
    loss_fn = nn.NLLLoss()                                         #损失函数
    optimizer = optim.Adam(net.parameters())                       #优化器
 
    last_acc = 0
    for epoch in range(300):
        train_acc = 0
        for xs,ys in train_loader:
            feature, cls = net(xs.to(device))
            #ys为tuple类型,不能直接传入cuda；标签需要为数字才能传入
            loss = loss_fn(torch.log(cls), ys.to(device))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            arg_max = torch.argmax(torch.log(cls), 1)
            # print(arg_max)
            train_acc += (arg_max == ys.to(device)).sum().item()
        train_mean_acc = train_acc/len(train_dataset)
        if train_mean_acc > last_acc:                                #精度上升时才保存参数
            print('epoch:{},train_mean_acc :{}'.format(epoch, train_mean_acc))
            #torch.save(net.state_dict(), save_path)
            #print("第 {} 轮参数保存成功！".format(epoch))
            print(str(epoch) + "Train_Loss      :" + str(loss.item()))
        else:
            print('epoch:{},train_mean_acc :{}'.format(epoch, train_mean_acc))
            print(str(epoch) + "Train_Loss      :" + str(loss.item()))
        last_acc = max(train_mean_acc, last_acc)
        
        test_acc = 0                                                 #求出验证集的平均精度
        for v_xs, v_ys in test_loader:
            v_feature, v_cls = net(v_xs.to(device))
            v_arg_max = torch.argmax(torch.log(v_cls), 1)
            test_acc += (v_arg_max == v_ys.to(device)).sum().item()
        test_mean_acc = test_acc/len(test_dataset)
        print("epoch:{}, test_mean_acc:{}".format(epoch, test_mean_acc))
        print()

epoch:0,train_mean_acc :0.058662384281126016
0Train_Loss      :39.807918548583984
epoch:0, test_mean_acc:0.06422364941443143

